In [1]:
import pandas as pd
import numpy as np
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
import folium
import json

In [2]:
yp_df = pd.read_excel("../data/moving_population/ingu_hour_yeungdeungpo.xlsx",sheet_name=0)

In [26]:
mp_df = pd.read_excel("..//data/moving_population/ingu_hour_mapo.xlsx",sheet_name=0)

In [13]:
ingu_var = [x for x in yp_df.columns if '_ingu' in x and len(x)>13]
print(len(ingu_var))
yp_df_2 = pd.melt(yp_df,['hour','latitude','longitude','gu_name','dong_name'],value_vars=ingu_var,var_name='성별연령대',value_name='인구')
yp_df_2.head()
yp_df_2['연령대'] = yp_df_2['성별연령대'].str.slice(stop=7)
yp_df_2['성별'] = yp_df_2['성별연령대'].str.slice(start=8)
yp_df_2.head()

18


,hour,latitude,longitude,gu_name,dong_name,성별연령대,인구,연령대,성별
0,0,37.510243,126.904651,영등포구,영등포본동,under10_male_ingu,0.185763,under10,male_ingu
1,0,37.516164,126.917051,영등포구,영등포본동,under10_male_ingu,0.112386,under10,male_ingu
2,0,37.512080,126.911425,영등포구,영등포본동,under10_male_ingu,0.000000,under10,male_ingu
3,0,37.511581,126.901812,영등포구,영등포본동,under10_male_ingu,0.417325,under10,male_ingu
4,0,37.514795,126.913667,영등포구,영등포본동,under10_male_ingu,0.076697,under10,male_ingu


In [ ]:
ingu_var = [x for x in yp_df.columns if '_ingu' in x and len(x)>13]
print(len(ingu_var))
yp_df_2 = pd.melt(yp_df,['hour','latitude','longitude','gu_name','dong_name'],value_vars=ingu_var,var_name='성별연령대',value_name='인구')
yp_df_2.head()
yp_df_2['연령대'] = yp_df_2['성별연령대'].str.slice(stop=7)
yp_df_2['성별'] = yp_df_2['성별연령대'].str.slice(start=8)
yp_df_2.head()

In [14]:
yp_df_2 = yp_df_2.drop("성별연령대", axis=1)
yp_df_3 

,hour,latitude,longitude,gu_name,dong_name,인구,연령대,성별
0,0,37.510243,126.904651,영등포구,영등포본동,0.185763,under10,male_ingu
1,0,37.516164,126.917051,영등포구,영등포본동,0.112386,under10,male_ingu
2,0,37.512080,126.911425,영등포구,영등포본동,0.000000,under10,male_ingu
3,0,37.511581,126.901812,영등포구,영등포본동,0.417325,under10,male_ingu
4,0,37.514795,126.913667,영등포구,영등포본동,0.076697,under10,male_ingu
...,...,...,...,...,...,...,...,...
3805663,23,37.495322,126.895154,영등포구,대림3동,0.203570,upper80,female_ingu
3805664,23,37.493985,126.897993,영등포구,대림3동,0.923317,upper80,female_ingu
3805665,23,37.500291,126.897376,영등포구,대림3동,0.568758,upper80,female_ingu
3805666,23,37.503882,126.894519,영등포구,대림3동,0.110731,upper80,female_ingu


In [21]:
yp_df_3 = yp_df_2.copy()
yp_df_3 = yp_df_3.apply(lambda row:row['성별'].split('_')[0],axis=1)

In [24]:
yp_df_2['성별'] = yp_df_3

In [28]:
yp_df_2.to_csv('post_processed_yp.csv')

In [30]:
from branca.colormap import linear

colormap = linear.YlGn_09.scale(
    yp_df_2['인구'].min(),
    yp_df_2['인구'].max())
colormap

In [82]:
groupby_dong = []
for _hour in range(24):
    temp_df = yp_df_2[yp_df_2['hour']==_hour].groupby('dong_name').sum()['인구'].to_frame()
    temp_df['dong_name'] = temp_df.index
    temp_df['hour'] = _hour
    temp_df['index'] = range(temp_df.shape[0])
    temp_df = temp_df.set_index('index')
    groupby_dong.append(temp_df)
    
move_groupby_dong = []
for x in groupby_dong:
    temp_df = x.copy()
    temp_df['인구'] = x['인구']-groupby_dong[3]['인구']
    move_groupby_dong.append(temp_df)


In [83]:
temp_df

,인구,dong_name,hour
index,,,
0,216.375021,당산1동,23
1,8.295235,당산2동,23
2,-483.624985,대림1동,23
3,-116.165969,대림2동,23
4,-27.835558,대림3동,23
5,-441.783650,도림동,23
6,-505.503337,문래동,23
7,-435.505765,신길1동,23
8,-275.096423,신길3동,23


In [40]:
geo_path = 'geodata/seoul_yp_mp_map.geojson'
geo_str = json.load(open(geo_path, encoding='utf-8'))
# geo_str

In [84]:
# 서울시 중심부의 위도, 경도 입니다. 
center = [37.541, 126.986] 
x,y = yp_df['latitude'].mean(),yp_df['longitude'].mean()
# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다. 
m = folium.Map(location=[x,y], zoom_start=13)

# # Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(geo_data=geo_str,
                  data=move_groupby_dong[2], columns=('dong_name','인구'), key_on='feature.properties.dong', fill_color=colormap, legend_name='10 인구수', ).add_to(m)

# 맵 m을 출력합니다. 

m

ValueError: color should be a string, not a <class 'branca.colormap.LinearColormap'>.